In [14]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R

In [2]:
(
    reference_x,
    reference_y,
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
    reference_x_4,
    reference_y_4,
) = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3 x_4 y_4")

f_interpolation = sympy.Matrix(sympy.symbols("f_1:5"))


# Get basis functions

In [3]:
monomial_basis = sympy.Matrix(
[
    1,
    reference_x,
    reference_y,
    reference_x*reference_y
 ]
)

In [4]:
Vander = sympy.Matrix(np.zeros((4, 4)))

for i, basis in enumerate(monomial_basis):

    v1 = {reference_x: 0, reference_y: 0}
    Vander[i, 0] = basis.subs(v1)

    v2 = {reference_x: 1, reference_y: 0}
    Vander[i, 1] = basis.subs(v2)

    v3 = {reference_x: 1, reference_y: 1}
    Vander[i, 2] = basis.subs(v3)
    
    v4 = {reference_x: 0, reference_y: 1}
    Vander[i, 3] = basis.subs(v4)

In [5]:
lagrange_bilinear_basis = Vander.inv() @ monomial_basis

# Get mapping function

In [6]:
monomial_basis = sympy.Matrix(
[
    1,
    reference_x,
    reference_y,
    reference_x*reference_y
 ]
)

V = sympy.zeros(4, 4)

for dofidx, basis in enumerate(monomial_basis):

    V[dofidx, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[dofidx, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[dofidx, 2] = basis.subs({reference_x: 1, reference_y: 1})
    V[dofidx, 3] = basis.subs({reference_x: 0, reference_y: 1})
    

mapping_basis = V.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * reference_x_1
    + mapping_basis[1] * reference_x_2
    + mapping_basis[2] * reference_x_3
    + mapping_basis[3] * reference_x_4
)
global_y = (
      mapping_basis[0] * reference_y_1
    + mapping_basis[1] * reference_y_2
    + mapping_basis[2] * reference_y_3
    + mapping_basis[3] * reference_y_4
)

mapping_function = sympy.Matrix([global_x, global_y])

In [7]:
J = mapping_function.jacobian([reference_x, reference_y]).inv()

In [8]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T

In [31]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

grad_u = u.jacobian([reference_x, reference_y])@J_cofactor_T
grad_v = v.jacobian([reference_x, reference_y])@J_cofactor_T

weak_form = grad_u@grad_v.T

### Calculate bilinear form and right values for forward pushed reference element

In [32]:
N = len(lagrange_bilinear_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = lagrange_bilinear_basis[idx]
    second = lagrange_bilinear_basis[jdx]

    weak_form_functional[idx][jdx] = weak_form.subs({u[0]:first, v[0]:second}).doit()
    weak_form_right_part[idx][jdx] = first * second

weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

weak_form_functional = weak_form_functional / J_cofactor_T.det()**2 * abs(J_cofactor_T.det())
weak_form_right_part = weak_form_right_part * abs(J_cofactor_T.det())

100%|██████████| 16/16 [00:00<00:00, 150.53it/s]


In [16]:
lambdify_symbols = [
    reference_x, reference_y,
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
    reference_x_4,
    reference_y_4,    
]

In [35]:
weak_form_right_part = weak_form_right_part @ f_interpolation
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part, cse=True)

In [36]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [37]:
quadrature_weights = np.array([
0.09754131164361921,
0.2117163509105210,
0.2255355359769118,
0.09730415294011353,
0.2255978016857150,
0.3510531956811132,
0.3511314245095946,
0.2118525411926204,
0.2116201646536030,
0.3511857026570127,
0.3512637749060175,
0.2256542961958117,
0.09746993474514315,
0.2257166005443878,
0.2117562971146869,
0.09723199711654983,
0.06609123516265450,
0.06607427278802323,
0.06606825552658292,
0.06605163800172888,
0.04798054519241257,
0.04797022666161702,
0.04807168904439760,
0.04806105514916250,
])

quadrature_weights /= quadrature_weights.sum()

quadrature_points = np.array([
[-0.1886381798247768, -0.9534228278198672],
[0.3158243867065065, -0.8124679583416120],
[0.7122535487614264, -0.5253420828029804],
[0.9536499381198605, -0.1884848209339622],
[-0.5255140441450369, -0.7118387124823607],
[-0.04156622116123301, -0.4250108457039333],
[0.4249386754351080, -0.04191684210258181],
[0.8124112175549880, 0.3156521043607226],
[-0.8126297846315392, -0.3155908346134177],
[-0.4247553230472783, 0.04140201954870253],
[0.04175248769766685, 0.4246831988441449],
[0.5251289118559497, 0.7121637731013759],
[-0.9534285320988584, 0.1886914057472521],
[-0.7117485896157119, 0.5253016177503731],
[-0.3154177460532097, 0.8125735341734832],
[0.1885390832384737, 0.9536564551709946],
[-0.8257630699887589, -0.9394679906605139],
[0.9394356990536500, -0.8259480185291852],
[-0.9395368212945202, 0.8256048988564928],
[0.8257904012838660, 0.9395040010720435],
[-0.9827093489403464, -0.6980866624366492],
[0.6978195696956143, -0.9827223639208844],
[-0.6983291127406627, 0.9825558709397986],
[0.9825693832248631, 0.6980615873134067],
])

quadrature_points[:, 0] = quadrature_points[:, 0] / 2 + 0.5
quadrature_points[:, 1] = quadrature_points[:, 1] / 2 + 0.5

In [38]:
def weak_form_functional_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4):
    
    return weak_form_functional_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4)@quadrature_weights

In [39]:
def weak_form_right_part_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, f_1, f_2, f_3, f_4):
    
    return weak_form_right_part_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, f_1, f_2, f_3, f_4)@quadrature_weights

In [40]:
interpolation_function = (lagrange_bilinear_basis.T @ f_interpolation)[0, 0]

interpolation_function_lambdified = sympy.lambdify(
    [*lambdify_symbols, *f_interpolation],
    interpolation_function,
    cse=True,
)

In [41]:
mapping_function_lambdified = sympy.lambdify(
    [*lambdify_symbols], mapping_function[:, 0], cse=True
)

### Save the results

In [44]:
dill.dump(
    weak_form_functional_lambdified_quadrature,
    open(
        "../calculations/bilinear_lagrange_weak_form_functional", "wb"
    ),
)

dill.dump(
    weak_form_right_part_lambdified_quadrature,
    open("../calculations/bilinear_lagrange_weak_form_right_part", "wb"),
)

dill.dump(
    interpolation_function_lambdified,
    open("../calculations/bilinear_lagrange_basis", "wb"),
)

dill.dump(
    mapping_function_lambdified,
    open("../calculations/bilinear_lagrange_mapping_function", "wb"),
)